In [1]:
import torch
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn as nn

In [2]:
torch.cuda.is_available()

True

In [ ]:
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(y_test,y_pred,labels=None,lenth=20,ratio=False,cmap="Blues",annot=True,width=12,fmt="g"):
    if labels == None:
        labels = pd.Series(labels for labels in y_pred.squeeze()).unique()
    conf_mat = pd.DataFrame(confusion_matrix(y_test.squeeze(), y_pred.squeeze(), labels=labels))
    if ratio == True:
        conf_mat = conf_mat.divide(conf_mat.sum(axis=1), axis=0)
    fig, ax = plt.subplots(figsize=(lenth, width))
    sns.heatmap(conf_mat,cmap=cmap,annot=annot,xticklabels=labels,yticklabels=labels,fmt=fmt)
    plt.ylabel("Actual")
    plt.xlabel("Predicted")
    plt.show()

In [3]:
f = open('word2idx_big.pkl','rb')
word2idx = pickle.load(f)
f.close()
f = open('idx2word_big.pkl','rb')
idx2word= pickle.load(f)
f.close()
f = open('df_big.pkl','rb')
word_df= pickle.load(f)
f.close()
f = open('embedding_big.pkl','rb')
word_embedding= pickle.load(f)
f.close()

In [4]:
EOS_NUM = 0
UNK_NUM = 1
PADDLING_NUM = 2

In [5]:
word_df['text'].apply(len).describe()

count    56821.000000
mean         9.087344
std          2.868494
min          0.000000
25%          7.000000
50%          9.000000
75%         11.000000
max         48.000000
Name: text, dtype: float64

In [6]:
word_df['text'].apply(len).describe()

count    56821.000000
mean         9.087344
std          2.868494
min          0.000000
25%          7.000000
50%          9.000000
75%         11.000000
max         48.000000
Name: text, dtype: float64

In [7]:
len(word_embedding)

355990

In [8]:
def getWordEmbedding(data):
    seq_embed = []
    _ = 0
    for word in data:
        seq_embed.append(word_embedding[word])
        _ += 1
        if _ == 9:
            break
    if _ <= 9:
        for i in range(9 - _):
            seq_embed.append(np.zeros(300))
    seq_embed = np.array(seq_embed)
    return seq_embed

In [9]:
# def getWordEmbedding(data):
#     seq_embed = []
#     _ = 0
#     for word in data:
#         seq_embed.append(word)
#         _ += 1
#         if _ == 9:
#             break
#     if _ <= 9:
#         for i in range(9 - _):
#             seq_embed.append()
#     seq_embed = np.array(seq_embed)
#     return seq_embed

In [10]:
word_df['text'] = word_df['text'].apply(getWordEmbedding)

In [11]:
train_df = word_df[['text','class']]
train_df['class'] = train_df['class'].apply(np.float32)

C:\Users\Javon\AppData\Local\Temp/ipykernel_21220/72822735.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['class'] = train_df['class'].apply(np.float32)


In [12]:
train_df['text'][0]

array([[ 0.77897102, -0.24356601, -0.26709101, ..., -0.371645  ,
         1.04136801,  0.18475001],
       [-0.005251  , -0.203603  , -0.764615  , ..., -0.31193799,
         0.56219399, -0.002608  ],
       [ 1.29011798, -0.39268801, -0.42361701, ...,  0.41950399,
         0.20449001,  0.64937598],
       ...,
       [ 0.079874  , -0.130665  ,  0.27537099, ...,  0.214256  ,
         0.33445001,  0.84352601],
       [-0.29280099,  0.449341  ,  0.14328299, ...,  0.15883499,
         0.67825699, -0.58762503],
       [ 0.38094899,  0.39534199, -0.105029  , ...,  0.35614899,
        -0.122519  ,  0.90290499]])

In [13]:
X_train,X_test,y_train,y_test = train_test_split(
    train_df['text'].values,
    train_df['class'].values,
    test_size=0.2,
    stratify=train_df['class'],
    random_state=42,
)

In [14]:
class seqDataset(torch.utils.data.Dataset):
    def __init__(self,data,label,transformer = None):
        super(seqDataset,self).__init__()
        self.data = data
        self.label = label
        self.transformer = transformer
    
    def __getitem__(self,idx):
        if self.transformer:
            return transformer(self.data[idx]),self.label[idx]
        return self.data[idx],self.label[idx]
    
    def __len__(self):
        return len(self.data)

In [28]:
X_train = [np.array(i) for i in X_train]
X_test = [np.array(i) for i in X_test]

In [29]:
X_train[0]

array([[ 0.533471, -0.738836, -0.745665, ...,  0.127822, -0.522074,
         0.193523],
       [-0.030646, -0.128678,  0.033818, ...,  0.174529, -0.073322,
         0.034218],
       [-0.365248, -0.336133, -0.045593, ...,  0.648002,  0.211448,
         0.375935],
       ...,
       [-0.445826, -0.097827,  0.230186, ...,  0.051419, -0.328975,
         0.01057 ],
       [-0.10139 , -0.143253, -0.035549, ...,  0.140243, -0.162254,
         0.017669],
       [-0.060803,  0.300508,  0.512175, ..., -0.168457, -0.568124,
         0.533321]], dtype=float32)

In [31]:
train_dataset = seqDataset(X_train,y_train)
test_dataset = seqDataset(X_test,y_test)

In [32]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=500)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=500)

In [33]:
for i,(x,y) in enumerate(train_loader):
    print(x.shape)
    break

torch.Size([500, 9, 300])


In [34]:
def train(model,EPOCH=150,learning_rate=0.05):
    model = model
    opt = torch.optim.SGD(params=model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()
    EPOCH = EPOCH
    itr = 0
    model.cuda()
    for epoch in range(EPOCH):
        for i,(x,y) in enumerate(train_loader):
            batch_X = torch.FloatTensor(np.float32(x)).cuda()
            batch_Y = torch.LongTensor(np.int64(y)).cuda()
            output = model(batch_X)
            opt.zero_grad()
            loss = loss_fn(output,batch_Y)
            loss.backward()
            opt.step()
            if (i+1) % 20 == 0:
                itr += 20
                acc = (batch_Y.detach().cpu().numpy() == output.detach().cpu().argmax(axis=1).numpy()).mean()
                print('epoch\t:{}\titr:{}\tloss:{}\tacc:{}\n'.format(epoch,itr,loss,acc))
                # print(output.detach().cpu())
    return model

def test(model):
    accs = []
    for i,(x,y) in enumerate(test_loader):
        batch_X = torch.FloatTensor(np.float32(x)).cuda()
        batch_Y = torch.LongTensor(np.int64(y)).cuda()
        output = model(batch_X)
        acc = (batch_Y.detach().cpu().numpy() == output.detach().cpu().argmax(axis=1).numpy()).mean()
        accs.append(acc)
    print(accs)

In [35]:
class SimpleRNN(nn.Module):
    def __init__(self,seq_len=9,embedding_size=300,h1_size=128,num_embeddings = len(idx2word)):
        super(SimpleRNN,self).__init__()
        self.embedding_size = embedding_size
        self.h1_size = h1_size
        self.seq_len = seq_len
        self.num_embeddings = num_embeddings

        # self.embed = nn.Embedding(num_embeddings=self.num_embeddings,embedding_dim=self.embedding_size)

        self.layer1 = nn.Sequential(
            nn.RNN(input_size = self.embedding_size,hidden_size = self.h1_size,num_layers=4),
        )

        self.output = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.seq_len*self.h1_size,10),
            nn.Softmax(),
        )
    def forward(self,x):
        # out = self.embed(x)
        out,hid = self.layer1(out)
        out = self.output(out)
        return out

In [36]:
class TransformerEncoder(nn.Module):
    def __init__(self,seq_len=9,embedding_size=300,num_embeddings = len(idx2word)):
        super(TransformerEncoder,self).__init__()
        self.num_embeddings = num_embeddings 
        self.embedding_size = embedding_size
        self.seq_len = seq_len
        self.emb = nn.Embedding(num_embeddings=self.num_embeddings,embedding_dim=self.embedding_size)
        self.layer1 = nn.Sequential(
            nn.TransformerEncoderLayer(d_model=embedding_size, nhead=10, batch_first=True),
            nn.TransformerEncoderLayer(d_model=embedding_size, nhead=10, batch_first=True),
            nn.TransformerEncoderLayer(d_model=embedding_size, nhead=10, batch_first=True),
            nn.TransformerEncoderLayer(d_model=embedding_size, nhead=10, batch_first=True),
        )

        self.output = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.seq_len*self.embedding_size,10),
            nn.Softmax(),
        )
    def forward(self,x):
        # out = self.emb(x)
        out = self.layer1(x)
        out = self.output(out)
        return out

In [37]:
class SimpleCNN(nn.Module):
    def __init__(self,embedding_size=300,num_embeddings = len(idx2word)):
        super(SimpleCNN,self).__init__()
        self.num_embeddings = num_embeddings
        self.embedding_size = embedding_size
        # self.embed = nn.Embedding(num_embeddings=self.num_embeddings,embedding_dim=self.embedding_size)
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels = 300,out_channels = 300,kernel_size=2),
            nn.ReLU(),
            nn.BatchNorm1d(300),
        )
        #[batch_size = 500,channels=128,seq_len - k_s + 1 = 8]
        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels = 300,out_channels = 200,kernel_size=2),
            nn.ReLU(),
            nn.BatchNorm1d(200),
        )
        #[batch_size = 500,embedding_size = 128,seq_len - k_s + 1 = 7]
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels = 200,out_channels = 100,kernel_size=3,stride = 2),
            nn.ReLU(),
            nn.BatchNorm1d(100),
            # nn.MaxPool1d(6),
        )
        #[batch_size = 500,embedding_size = 256,(seq_len - k_s)/2+ 1 = 3]
        self.conv4 = nn.Sequential(
            nn.Conv1d(in_channels = 100,out_channels = 100,kernel_size = 3),
            nn.ReLU(),
            nn.BatchNorm1d(100),
        )
        #[batch_size = 500,embedding_size = 256,seq_len - k_s + 1 = 1]

        self.output = nn.Sequential(
            nn.Flatten(),
            nn.Linear(100,10),
            nn.Softmax()
        )
    def forward(self,x):
        # out = self.embed(x).transpose(2,1)
        out = self.conv1(x.transpose(2,1))
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.output(out)
        return out

In [38]:
model = train(TransformerEncoder())
test(model)

D:\Anaconda\envs\pytorch\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


epoch	:0	itr:20	loss:2.119532585144043	acc:0.374

epoch	:0	itr:40	loss:2.013864040374756	acc:0.468

epoch	:0	itr:60	loss:1.9531245231628418	acc:0.544

epoch	:0	itr:80	loss:1.8542976379394531	acc:0.654

epoch	:1	itr:100	loss:1.8420692682266235	acc:0.648

epoch	:1	itr:120	loss:1.838194727897644	acc:0.648

epoch	:1	itr:140	loss:1.832452654838562	acc:0.648

epoch	:1	itr:160	loss:1.7865103483200073	acc:0.694

epoch	:2	itr:180	loss:1.7969391345977783	acc:0.674

epoch	:2	itr:200	loss:1.783646821975708	acc:0.702

epoch	:2	itr:220	loss:1.7917819023132324	acc:0.674

epoch	:2	itr:240	loss:1.7515790462493896	acc:0.728

epoch	:3	itr:260	loss:1.7646020650863647	acc:0.708

epoch	:3	itr:280	loss:1.7493075132369995	acc:0.724

epoch	:3	itr:300	loss:1.7680319547653198	acc:0.694

epoch	:3	itr:320	loss:1.7253509759902954	acc:0.754

epoch	:4	itr:340	loss:1.7451316118240356	acc:0.726

epoch	:4	itr:360	loss:1.7425172328948975	acc:0.72

epoch	:4	itr:380	loss:1.7508376836776733	acc:0.72

epoch	:4	itr:400	loss:1

In [ ]:
model = train(SimpleCNN())
test(model)

In [ ]:
model = train(SimpleRNN())
test(model)